# **Proyecto TB1: Asignación de Tripulaciones Aéreas con CSP**

**Tópicos en Ciencias de Computación - 2025-2**

- **Integrantes:**
  - Ibrahim Imanol Jordi Arquiñigo Jacinto
  - Ian Joaquin Sanchez Alva
  - Eduardo Jose Rivas Siesquen
  - Daniel Orlando Luis Lázaro

## Introducción al Problema

Este notebook contiene el modelo y la solución completa para el problema de **Asignación de Tripulaciones Aéreas**. El objetivo es asignar 20 empleados a 10 vuelos, satisfaciendo un conjunto de reglas complejas (tamaño de tripulación, roles, idiomas y descanso).

El problema se modela como un **Problema de Satisfacción de Restricciones (CSP)** y se resuelve con el solver **CP-SAT** de Google OR-Tools.

In [ ]:
!pip install ortools

   ---------------------------------------- 0.0/20.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/20.5 MB ? eta -:--:--
    --------------------------------------- 0.3/20.5 MB ? eta -:--:--
   -- ------------------------------------- 1.0/20.5 MB 4.2 MB/s eta 0:00:05
   ------ --------------------------------- 3.1/20.5 MB 7.4 MB/s eta 0:00:03
   ------ --------------------------------- 3.4/20.5 MB 4.8 MB/s eta 0:00:04
   ---------- ----------------------------- 5.2/20.5 MB 6.2 MB/s eta 0:00:03
   ------------- -------------------------- 6.8/20.5 MB 6.0 MB/s eta 0:00:03
   ------------------ --------------------- 9.7/20.5 MB 7.4 MB/s eta 0:00:02
   ----------------------- ---------------- 12.1/20.5 MB 7.9 MB/s eta 0:00:02
   -------------------------- ------------- 13.6/20.5 MB 8.1 MB/s eta 0:00:01
   -------------------------------- ------- 16.8/20.5 MB 8.7 MB/s eta 0:00:01
   ------------------------------------- -- 19.1/20.5 MB 8.9 MB/s eta 0:00:01
   -------------

In [1]:
# Importar la biblioteca necesaria
from ortools.sat.python import cp_model

## 1. Definición de Datos del Problema

En esta sección, definimos todas las constantes y datos de entrada: listas de empleados, requisitos de los vuelos y las habilidades de idioma de cada persona.

In [ ]:
# --- 1. Definición de Datos ---

stewards = ["Tom", "David", "Jeremy", "Ron", "Joe", "Bill", "Fred", "Bob", "Mario", "Ed"]
hostesses = ["Carol", "Janet", "Tracy", "Marilyn", "Carolyn", "Cathy", "Inez", "Jean", "Heather", "Juliet"]
all_employees = stewards + hostesses

num_employees = len(all_employees)
num_flights = 10

employee_to_idx = {name: i for i, name in enumerate(all_employees)}
idx_to_employee = {i: name for i, name in enumerate(all_employees)}

crew_size = [4, 5, 5, 6, 7, 4, 5, 6, 6, 7]
min_hostesses = [1, 1, 1, 2, 3, 1, 1, 1, 2, 3]
min_stewards = [1, 1, 1, 2, 3, 1, 1, 1, 2, 3]

speaks_french = {"Inez", "Bill", "Jean", "Juliet"}
speaks_spanish = {"Tom", "Jeremy", "Mario", "Cathy", "Juliet"}
speaks_german = {"Bill", "Fred", "Joe", "Mario", "Marilyn", "Inez", "Heather"}

## 2. Modelado del CSP y Función de Resolución

A continuación, se define la función principal que:
1.  Crea el modelo CSP (`CpModel`).
2.  Define las variables de decisión (la matriz de asignación).
3.  Aplica todas las restricciones del problema.
4.  Invoca al solver para encontrar una solución.
5.  Imprime los resultados de forma clara y legible.

In [ ]:
def resolver_asignacion_tripulacion():
    """
    Modela y resuelve el problema de asignación de tripulaciones
    como un Problema de Satisfacción de Restricciones (CSP).
    """
    
    model = cp_model.CpModel()

    assignments = {}
    for e in range(num_employees):
        for v in range(num_flights):
            assignments[(e, v)] = model.NewBoolVar(f"assign_emp{e}_flight{v}")

    for v in range(num_flights):
        model.Add(sum(assignments[(e, v)] for e in range(num_employees)) == crew_size[v])

        hostess_indices = [employee_to_idx[h] for h in hostesses]
        model.Add(sum(assignments[(e, v)] for e in hostess_indices) >= min_hostesses[v])

        steward_indices = [employee_to_idx[s] for s in stewards]
        model.Add(sum(assignments[(e, v)] for e in steward_indices) >= min_stewards[v])

        french_speakers = [employee_to_idx[name] for name in speaks_french]
        model.Add(sum(assignments[(e, v)] for e in french_speakers) >= 1)

        spanish_speakers = [employee_to_idx[name] for name in speaks_spanish]
        model.Add(sum(assignments[(e, v)] for e in spanish_speakers) >= 1)

        german_speakers = [employee_to_idx[name] for name in speaks_german]
        model.Add(sum(assignments[(e, v)] for e in german_speakers) >= 1)
        
    for e in range(num_employees):
        for v in range(num_flights - 2):
            model.AddImplication(assignments[(e, v)], assignments[(e, v + 1)].Not())
            model.AddImplication(assignments[(e, v)], assignments[(e, v + 2)].Not())

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("="*40)
        print(" Asignación de Tripulación Encontrada ")
        print("="*40 + "\n")
        for v in range(num_flights):
            flight_title = f"--- Vuelo {v+1} (Requiere {crew_size[v]} tripulantes) ---"
            print(flight_title)
            
            crew_list = []
            for e in range(num_employees):
                if solver.Value(assignments[(e, v)]) == 1:
                    crew_list.append(idx_to_employee[e])
            
            print(f"Tripulación Asignada: {', '.join(crew_list)}")
            print("-" * len(flight_title) + "\n")
    else:
        print("No se encontró una solución factible para el problema.")

## 3. Ejecución del Modelo

La siguiente celda ejecuta la función `resolver_asignacion_tripulacion()` para encontrar y mostrar la solución. La salida que se muestra a continuación es un cronograma completo y válido que cumple con todas las reglas.